#Basic setups

In [ ]:
import random
import re
import math
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional
import numpy as np

In [ ]:
class NGramLanguageModel:
  def __init__(self, n: int = 2):
    """
    Initialize n-gram lang model
    n: order of the n-gram (2 is the bigram, 3 the trigram and so on)
    """
    self.n = n
    self.ngram_counts = defaultdict(int)
    self.context_counts = defaultdict(int)
    self.vocabulary = set()
    self.vocab_size = 0


  def tokenize(self, text:str) -> List[str]:
    """
    Basic tokenization.
    Converts to lowercase and split on whitespaces.
    """
    tokens = re.findall(r'\b\w+\b|[.!?]', text.lower())
    return tokens

  def add_sentence_markers(self, tokens: List[str]) -> List[str]:
    """
    Add start and end sentence markers
    """
    start_markers = ['<s>'] * (self.n - 1)
    print("start_markers", start_markers, "\n")
    end_markers = ['</s>']
    return start_markers + tokens + end_markers



  #------------- Training the Model (Counting N-grams) -------------
  def train(self, corpus: List[str]):
    """
    Train the n-gram model on a corpus of sentences
    """
    print(f"Training: {self.n}-gram model")

    for sentence in corpus:
      tokens = self.tokenize(sentence)
      tokens_with_markers = self.add_sentence_markers(tokens)
      print("tokens_with_markers:", tokens_with_markers, "\n")

      # Update vocabulary
      self.vocabulary.update(tokens) # set.update applies union

      # Count n-grams
      for i in range(len(tokens_with_markers) - self.n + 1):
        ngram = tuple(tokens_with_markers[i:i + self.n])
        print("ngram: ", ngram, "\n")
        context = ngram[:-1] ## All but the last word

        self.ngram_counts[ngram] += 1
        self.context_counts[context] += 1


      self.vocab_size = len(self.vocabulary)
      print(f"Vocabulary size |V| = {self.vocab_size}", "\n")
      print(f"total n-grams: {len(self.ngram_counts)}", "\n")




  #------------- Maximum Likelihood Estimation -------------
  def get_mle_probability(self, word: str, context: Tuple[str]) -> float:
    """Calculate the Maximum Likelihood Estimated probability"""
    ngram = context + (word,) # Concatenate context tuple with current word tuple to form complete n-gram
    print("ngram:", ngram, "\n")

    if self.context_counts[context] ==0:
      return 0.0

    print("self.ngram_counts[ngram]:", self.ngram_counts[ngram], "\n")
    print("self.context_counts[ngram]:", self.context_counts[ngram], "\n")

    return self.ngram_counts[ngram] / self.context_counts[context]


  def get_sentence_probability(self, sentence: str) -> float:
    """ Calculate probability of entire sentence using the chain rule"""
    tokens = self.tokenize(sentence)
    tokens_with_markers = self.add_sentence_markers(tokens)

    log_prob = 0.0

    for i in range(self.n - 1, len(tokens_with_markers)):
      context = tuple(tokens_with_markers[i - self.n + 1 : i])
      print("context: ", context, "\n")

      word = tokens_with_markers[i]
      print("word: ", word, "\n")

      prob = self.get_mle_probability(word, context)
      if prob == 0:
        return 0.0 # Prob for any n-gram unseen

      log_prob += math.log(prob)


    return math.exp(log_prob)




  #------------- Add-One (Laplace) Smoothing -------------

  def get_smoothed_probability(self, word: str, context: Tuple[str]) -> float:
    """ Calculate the probability with the Add-1 (Laplace) smoothing """

    ngram = context + (word,)


    # Add-1 smoothing formula: (count + 1) / (context_count + |V|) with |V| = Vocabulary size
    numerator = self.ngram_counts[ngram] + 1
    denominator = self.context_counts[context] + self.vocab_size

    print("numerator:", numerator, "\n")
    print("denominator:", denominator, "\n")
    print("numerator/denominator = ", numerator/denominator, "\n")

    return(numerator/denominator)


  def get_smoothed_sentence_probability(self, sentence: str) -> float:
    """ Calculate the smoothed probability for an entire sentence"""
    tokens = self.tokenize(sentence)
    tokens_with_markers = self.add_sentence_markers(tokens)

    log_prob = 0.0

    for i in range(self.n - 1, len(tokens_with_markers)):
      context = tuple(tokens_with_markers[i - self.n + 1: i])
      word = tokens_with_markers[i]

      prob = self.get_smoothed_probability(word, context)
      log_prob += math.log(prob)

    return(math.exp(log_prob))



# ------------- Perplexity Calculation -------------

  def calculate_perplexity(self, test_sentences: List[str], smoothed: bool = True) -> float:
    """ Calculate perplexity on the test set """
    total_log_prob = 0.0
    total_words = 0

    for sentence in test_sentences:
      tokens = self.tokenize(sentence)
      tokens_with_markers = self.add_sentence_markers(tokens)

      # Count actual words (not including start markers, but including end marker)
      total_words += len(tokens) + 1 # for the </s> marker

      for i in range(self.n - 1, len(tokens_with_markers)):
        context = tuple(tokens_with_markers[i - self.n + 1 : i])
        print("context: ", context, "\n")

        word = tokens_with_markers[i]
        print("word: ", word, "\n")

        if smoothed:
          prob = self.get_smoothed_probability(word, context)

        else:
          prob = self.get_mle_probability(word, context)

          if prob == 0:
            return float('inf') # Infinite perplexity for zero probability


        total_log_prob += math.log(prob)


    """
    Perplexity is both:
    Negative + exp: 2^(-avg_log_prob) or exp(-avg_log_prob)

    Perplexity is 2 to the power of the negative average log probability
    Or also, Perplexity = exp(-1/N * sum(log(P(w_i))))

    PPL could be from any base, but with 2 we have it in bits, which is very convenient
    Common choices:
    perplexity = 2^(-avg_log_prob)     # Base 2 → measured in bits
    perplexity = e^(-avg_log_prob)     # Base e → natural log (most common in practice)


    Average log prob: Normalizes total probability by sentence length (per-word measure)

    Lower probability = higher perplexity (inverse relationship)

    """

    avg_log_prob = total_log_prob / total_words
    print(f"avg_log_prob = total_log_prob / total_words: {avg_log_prob} = {total_log_prob} / {total_words} " )

    perplexity = math.exp(-avg_log_prob)

    return(perplexity)


# ------------- Text Generation (Sampling) -------------

  def generate_sentence(self, max_length: int = 20, smoothed: bool = True) -> str:
    """ Generate a sentence by sampling from the language model """

    # Start with initial context
    current_context = ['<s>'] * (self.n -1)
    sentence = []

    for _ in range(max_length):
      context_tuple = tuple(current_context[-(self.n - 1):])

      print("context_tuple: ", context_tuple, "\n" )

      # Get all possible next words and their probabilities
      candidates = []
      probabilities = []

      # Considering all words in vocabulary included the marker </s>
      for word in list(self.vocabulary) + ['</s>']:
        if smoothed:
          prob = self.get_smoothed_probability(word, context_tuple)

        else:
          prob = self.get_mle_probability(word, context_tuple)

        if prob > 0:
          candidates.append(word)
          probabilities.append(prob)

        # Normalizing probabilities
        total_prob = sum(probabilities)
        probabilities = [p / total_prob for p in probabilities]

        print("candidates: ", str([i for i in enumerate(candidates)]),"\n")
        print("probabilities: ", str([i for i in enumerate(probabilities)]),"\n")

        #Sampling next word
        next_word = np.random.choice(candidates, p=probabilities)

        if next_word == '</s>':
          break

        sentence.append(next_word)
        current_context.append(next_word)


    return ' '.join(sentence)

# -------------  Complete Example with Analysis -------------

  def analyze_model(self):
    """Print model statistics and examples"""
    print(f"\n=== {self.n}-gram Model Analysis ===")
    print(f"Vocabulary size: {self.vocab_size}")
    print(f"Total n-gram types: {len(self.ngram_counts)}")
    print(f"Total n-gram tokens: {sum(self.ngram_counts.values())}")

    # Most frequent n-grams
    print(f"\nMost frequent {self.n}-grams:")
    for ngram, count in sorted(self.ngram_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
      print(f"  {' '.join(ngram)}: {count}")

    # Example probabilities
    if self.n == 2:  # Bigram examples
      print(f"\nExample bigram probabilities:")
      examples = [('i', 'want'), ('want', 'to'), ('to', 'eat')]
      for context, word in examples:
        mle_prob = self.get_mle_probability(word, (context,))
        smooth_prob = self.get_smoothed_probability(word, (context,))
        print(f"  P({word}|{context}): MLE={mle_prob:.4f}, Smoothed={smooth_prob:.4f}")



In [ ]:
# Example
def main():
    # Sample corpus
    corpus = [
        "I am Sam",
        "Sam I am",
        "I am Sam",
        "I do not like green eggs and ham",
        "I want to eat food",
        "I want Chinese food",
        "I want to eat Chinese food",
        "Sam wants to eat",
        "Sam likes green eggs"
    ]

    print("=== N-gram Language Model Demo ===\n")

    # Train bigram model
    bigram_model = NGramLanguageModel(n=2)
    bigram_model.train(corpus)
    bigram_model.analyze_model()

    # Test sentences
    test_sentences = [
        "I want Chinese food",
        "Sam likes eggs",
        "I eat green ham"  # This should be less probable
    ]

    print(f"\n=== Sentence Probabilities ===")
    for sentence in test_sentences:
        mle_prob = bigram_model.get_sentence_probability(sentence)
        smooth_prob = bigram_model.get_smoothed_sentence_probability(sentence)
        print(f"'{sentence}':")
        print(f"  MLE probability: {mle_prob:.2e}")
        print(f"  Smoothed probability: {smooth_prob:.2e}")

    # Calculate the perplexity
    perplexity = bigram_model.calculate_perplexity(test_sentences, smoothed=True)
    print(f"\nPerplexity on test set: {perplexity:.2f}")

    # Generate some sentences
    print(f"\n=== Generated Sentences ===")
    for i in range(5):
        generated = bigram_model.generate_sentence(smoothed=True)
        print(f"  {i+1}: {generated}")

    # Compare with trigram model
    print(f"\n=== Trigram Model Comparison ===")
    trigram_model = NGramLanguageModel(n=3)
    trigram_model.train(corpus)

    trigram_perplexity = trigram_model.calculate_perplexity(test_sentences, smoothed=True)
    print(f"Bigram perplexity: {perplexity:.2f}")
    print(f"Trigram perplexity: {trigram_perplexity:.2f}")

if __name__ == "__main__":
    main()

=== N-gram Language Model Demo ===

Training: 2-gram model
start_markers ['<s>'] 

tokens_with_markers: ['<s>', 'i', 'am', 'sam', '</s>'] 

ngram:  ('<s>', 'i') 

ngram:  ('i', 'am') 

ngram:  ('am', 'sam') 

ngram:  ('sam', '</s>') 

Vocabulary size |V| = 3 

total n-grams: 4 

start_markers ['<s>'] 

tokens_with_markers: ['<s>', 'sam', 'i', 'am', '</s>'] 

ngram:  ('<s>', 'sam') 

ngram:  ('sam', 'i') 

ngram:  ('i', 'am') 

ngram:  ('am', '</s>') 

Vocabulary size |V| = 3 

total n-grams: 7 

start_markers ['<s>'] 

tokens_with_markers: ['<s>', 'i', 'am', 'sam', '</s>'] 

ngram:  ('<s>', 'i') 

ngram:  ('i', 'am') 

ngram:  ('am', 'sam') 

ngram:  ('sam', '</s>') 

Vocabulary size |V| = 3 

total n-grams: 7 

start_markers ['<s>'] 

tokens_with_markers: ['<s>', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] 

ngram:  ('<s>', 'i') 

ngram:  ('i', 'do') 

ngram:  ('do', 'not') 

ngram:  ('not', 'like') 

ngram:  ('like', 'green') 

ngram:  ('green', 'eggs') 

ngram: 

In [ ]:
class InterpolatedNGramModel:
  """N-gram model with  linear interpolation smoothing"""

  def __init__(self, max_n: int = 3):
    self.max_n = max_n
    self.models = {}
    self.lambdas = {}

    # Create models of different orders
    for n in range(1, max_n + 1):
      self.models[n] = NGramLanguageModel(n)

  def train(self, corpus: List[str], held_out_corpus: List[str] = None):
    """Train all the n-gram models"""
    for n in range(1, self.max_n + 1):
      self.models[n].train(corpus)

    # Set equal weights (in practice, these would be optimized on held-out data)
    total_weight = sum(range(1, self.max_n + 1))
    for n in range(1, self.max_n + 1):
      self.lambdas[n] = n / total_weight  # Higher weight for higher-order models

  def get_interpolated_probability(self, word: str, context: List[str]) -> float:
    """Calculate the interpolated probability"""
    prob = 0.0

    for n in range(1, self.max_n + 1):
      # Adjusting context length for this n-gram order
      if n == 1:
        n_context = ()
      else:
        n_context = tuple(context[-(n-1):])

      n_prob = self.models[n].get_smoothed_probability(word, n_context)
      prob += self.lambdas[n] * n_prob

    return prob


# Sample corpus
corpus = [
    "I am Sam",
    "Sam I am",
    "I am Sam",
    "I do not like green eggs and ham",
    "I want to eat food",
    "I want Chinese food",
    "I want to eat Chinese food",
    "Sam wants to eat",
    "Sam likes green eggs"
]

# Example
interpolated_model = InterpolatedNGramModel(max_n=3)
interpolated_model.train(corpus)

# Testing the interpolated probabilities
context = ['I', 'want']
word = 'food'
interp_prob = interpolated_model.get_interpolated_probability(word, context)
print(f"Interpolated P({word}|{' '.join(context)}): {interp_prob:.4f}")

Training: 1-gram model
start_markers [] 

tokens_with_markers: ['i', 'am', 'sam', '</s>'] 

ngram:  ('i',) 

ngram:  ('am',) 

ngram:  ('sam',) 

ngram:  ('</s>',) 

Vocabulary size |V| = 3 

total n-grams: 4 

start_markers [] 

tokens_with_markers: ['sam', 'i', 'am', '</s>'] 

ngram:  ('sam',) 

ngram:  ('i',) 

ngram:  ('am',) 

ngram:  ('</s>',) 

Vocabulary size |V| = 3 

total n-grams: 4 

start_markers [] 

tokens_with_markers: ['i', 'am', 'sam', '</s>'] 

ngram:  ('i',) 

ngram:  ('am',) 

ngram:  ('sam',) 

ngram:  ('</s>',) 

Vocabulary size |V| = 3 

total n-grams: 4 

start_markers [] 

tokens_with_markers: ['i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] 

ngram:  ('i',) 

ngram:  ('do',) 

ngram:  ('not',) 

ngram:  ('like',) 

ngram:  ('green',) 

ngram:  ('eggs',) 

ngram:  ('and',) 

ngram:  ('ham',) 

ngram:  ('</s>',) 

Vocabulary size |V| = 10 

total n-grams: 11 

start_markers [] 

tokens_with_markers: ['i', 'want', 'to', 'eat', 'food', '</s>'] 

